# Assignment 2b Notebook: Automated Design of AI Agents with Genetic Programming
In this assignment, you will further iterate on to your Assignment 2a implementation of random parse tree generation to realize a full genetic programming (GP) implementation. Like Assignment Series 1, you will leverage components you've already implemented in this assignment. From your previous assignment implementations, copy over the following files:
* baseEvolution.py
* selection.py
* treeGenotype.py

As usual, be careful not to overwrite any of the provided files that may have been modified since previous assignments. To begin the assignment, execute the following cell. **If you implemented your genotype in a new file, be sure to import it in the next cell!**

In [ ]:
# Configure this notebook to automatically reload modules as they're modified
# https://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
from snakeeyes import readConfig
from fitness import play_GPac
from selection import *
from 

%matplotlib inline
plt.rcParams['figure.figsize'] = (12.0, 12.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

print('The first cell has been executed!')

## Recombination
Assuming you've correctly implemented population initialization in Assignment 2a, we'll begin this assignment by implementing recombination with subtree crossover. Like Assignment Series 1, your recombination method should take a mate as input and recombine the genes of `self` and `mate`, assign the recombination to the `gene` member variable of `child` and then return child. Recall that you should somehow enforce a configurable maximum tree depth throughout this assignment and this applies to children produced with recombination and mutation.

How you implement subtree crossover in practice depends on your implementation of the parse tree gene and is thus open-ended. Implement `treeGenotype.recombine` and test your implementation by executing the following cell.

In [ ]:
# read config
config = readConfig('./configs/green2b_config.txt', globalVars=globals(), localVars=locals())

# initialize population
randomPopulation = treeGenotype.initialization(25, **config['initialization_kwargs'])

# perform recombination
children = list()
for idx in range(len(randomPopulation)):
    children.append(randomPopulation[idx].recombine(randomPopulation[idx+1%len(randomPopulation)], **config['recombination_kwargs']))

# print recombined trees to files
for idx, individual in enumerate(children):
    with open(f'treeTests/tree{idx}r.txt','w') as f:
        f.write(individual.print())
del randomPopulation
del children
del config

# evaluate tree files
!python treeCheck.py treeTests/tree*r.txt

## Mutation
Recall that mutation in GP is mutually exclusive with recombination. That is to say that mutation in GP is used to directly produce children by mutating a copy of the parent.

Using your parse tree gene implementation, implement subtree mutation in `treeGenotype.mutate` and test your implementation by executing the next cell. Don't forget to enforce max tree depth like in recombination!

In [ ]:
# read config
config = readConfig('./configs/green2b_config.txt', globalVars=globals(), localVars=locals())

# initialize population
randomPopulation = treeGenotype.initialization(25, **config['initialization_kwargs'])

# perform mutation
children = list()
for idx in range(len(randomPopulation)):
    children.append(randomPopulation[idx].mutate(**config['mutation_kwargs']))

# print mutated trees to files
for idx, individual in enumerate(children):
    with open(f'treeTests/tree{idx}m.txt','w') as f:
        f.write(individual.print())
del randomPopulation
del children
del config

# evaluate tree files
!python treeCheck.py treeTests/tree*m.txt

## Implementing Genetic Programming
By this point, you should have a complete implementation of your genotype and we can turn our attention to implementing the complete GP algorithm.

### Parsimony Pressure
Recall from the lecture and videos by Dr. Koza that GP tends to produce individual genotypes of increasing size without a mechanism to curb this. You will implement a parsimony penalty to encourage your GP to produce more compact trees (in addition to the depth limits you already have). With this mechanism of parsimony pressure, you will penalize the fitness of a solution based on their size. $fitness(i)=rawFitness(i)-C_p*size(i)$ where $C_p$ is a penalty coefficient from your config file and $size(i)$ is a function that returns the size of individual $i$. The two most obvious metrics of tree size are tree depth and node count, but you are encouraged with experiments to use the size metric that performs best.

**Note**: The implementation of a parsimony penalty is nearly identical to the penalty-based constraint satisfaction method you implemented in Assignment 1c. Like Assignment 1c, *it is not meaningful to compare penalized fitness with unpenalized fitness* and we require that you use penalized fitness for evolution and raw fitness for analysis.

In the following cell, implement a function that performs fitness evaluations on an input population using the input configuration parameters and assigns parsimony-penalized fitness, raw fitness, and saves the log of the fitness evaluation.

In [ ]:
# evaluate the population and assign fitness, rawFitness, and logs as described above
def evaluate_population(population, penalty_coefficient, **fitness_kwargs):
    pass

Now test your implementation by executing the following cell.

In [ ]:
config = readConfig('./configs/green2b_config.txt', globalVars=globals(), localVars=locals())

# calling your function to test things out (this function is called the same as in notebook 1c)
evaluate_population(examplePopulation, **config['fitness_kwargs'])

print(f'Individuals with unassigned fitness: {len([individual.fitness for individual in examplePopulation if individual.fitness is None])}')
print(f'Number of fitness evaluations performed: {len([individual.fitness for individual in examplePopulation if individual.fitness is not None])}')
print(f'Average fitness of population: {statistics.mean([individual.fitness for individual in examplePopulation])}')
maxFitness = max([individual.fitness for individual in examplePopulation])
print(f'Best fitness in population: {maxFitness}')
print(f'Average unpenalized (raw) fitness of population: {statistics.mean([individual.rawFitness for individual in examplePopulation])}')
maxRawFitness = max([individual.rawFitness for individual in examplePopulation])
print(f'Best unpenalized (raw) fitness in population: {maxRawFitness}')
bestLog = None
for individual in examplePopulation:
    if individual.fitness == maxFitness:
        bestLog = individual.log
        break

print(f'Found log of highest scoring individual? {bestLog is not None}')
with open(game_log_path, 'w') as f:
	[f.write(f'{line}\n') for line in bestLog]
    
print(f"The log of the most fit individual was written to {game_log_path}")

del examplePopulation
del config

### Child Generation
With fitness evaluation implemented, we can now perform parent selection and move to implementing child generation with the `geneticProgrammingPopulation` class. This class inherits the `baseEvolutionPopulation` class from last assignment and should be able to directly use the inherited initialization and survival selection methods without modification. Child generation is different in GP, as mentioned previously in this notebook, so you need to implement the GP version of `generate_children` in the `geneticProgrammingPopulation` class. Once complete, test your implementation in the following cell.

In [ ]:
config = readConfig('./configs/green2b_config.txt', globalVars=globals(), localVars=locals())

# full initialization of your GP population
exampleEA = geneticProgrammingPopulation(**config['EA_configs'], **config)
evaluate_population(exampleEA.population, **config['fitness_kwargs'])
exampleEA.evaluations = len(exampleEA.population)
print(f'Average fitness of population: {statistics.mean([individual.fitness for individual in exampleEA.population])}')
print(f'Best fitness in population: {max([individual.fitness for individual in exampleEA.population])}')
print(f'Number of fitness evaluations: {exampleEA.evaluations}')

# generate children
children = exampleEA.generate_children()
evaluate_population(children, config['fitness_kwargs'])
exampleEA.evaluations += len(children)
print(f'Average fitness of children: {statistics.mean([individual.fitness for individual in children])}')
print(f'Best fitness of children: {max([individual.fitness for individual in children])}')
print(f'Number of fitness evaluations: {exampleEA.evaluations}')

# print children trees to files
for idx, individual in enumerate(children):
    with open(f'treeTests/tree{idx}c.txt','w') as f:
        f.write(individual.print())
del exampleEA
del children
del config

# evaluate tree files
!python treeCheck.py treeTests/tree*c.txt

## Single Run Experiment
At this point, you should have implemented the full GP algorithm to evolve GPac controllers! Now put all the components together and implement a single-run experiment with 2,000 fitness evaluations in the next cell.

In [ ]:
number_evaluations = 2000

# You can parse different configuration files here as necessary
config = readConfig('./configs/green2b_config.txt', globalVars=globals(), localVars=locals())

# implement your EA here


## Multi-Run Experiments
Implement a full 30-run experiment with 2,000 fitness evaluations per run. For each generation, log the average fitness and best fitness of the current population. Average this data across all 30 runs to produce a plot of fitness vs evaluations with average and best fitness averaged across all 30 runs. For each run, log the best (unpenalized/raw) fitness encountered for statistical analysis with your results from Assignment 2a. For the individual with the highest fitness of the full 30-run experiment, save the log for visualization and informal analysis and comparison with the best agent from Assignment 2a.

In [ ]:
number_runs = 30
number_evaluations = 2000

# You can parse different configuration files here as necessary
config = readConfig('./configs/green2b_config.txt', globalVars=globals(), localVars=locals())

# Implement your 30-run experiment here


## RED Deliverables
The RED deliverables for this assignment task you with implementing controllers that control multiple Pac-Man or multiple ghosts. Implementing this requires the modification of your primitives, alternate definitions of fitness (if evolving ghosts), and a potentially large modification of the fitness function `play_GPac`. To make the modifications to `play_GPac` more feasible, we've included an updated stub function in your repo. In case you've overwritten this file with your code from Assignment 2a, here is the updated function definition
```python
def play_GPac(pac_controller, map_gene, height, width, ghost_type='wander', ghost_controller=None, pac_type='genetic', **kwargs):
	'''	Fitness function that plays a game using the provided pac_controller,
		map_gene, and/or ghost_controller to assess all inputs.

		Returns Pac-Man score from a full game as well as the game log.
	'''
	game_map = translate_gene(map_gene, height, width)
	game_map, num_repairs = repair_map(game_map)
	game = gpac.GPacGame(game_map, **kwargs)

	# select Pac-Man strategy
	if pac_type == 'pill':
		pac_class = staticAgents.shortestPathPillAgent
		pac = {player: pac_class() for player in game.players if 'm' in player}
	elif pac_type == 'fruit':
		pac_class = staticAgents.shortestPathFruitAgent
		pac = {player: pac_class() for player in game.players if 'm' in player}
	elif pac_type == 'avoid':
		pac_class = staticAgents.AvoidingPacmanAgent
		pac = {player: pac_class() for player in game.players if 'm' in player}
	elif pac_type != 'genetic':
		raise ValueError(f"{pac_type} is not a known type of Pac-man agent. Accepted types are ['pill','fruit','avoid','genetic']")

	# select ghost strategy
	if ghost_type == 'wander':
		ghost_class = staticAgents.RandomGhostAgent
		ghosts = {player: ghost_class() for player in game.players if 'm' not in player}
	elif ghost_type == 'chase':
		ghost_class = staticAgents.ChasingGhostAgent
		ghosts = {player: ghost_class() for player in game.players if 'm' not in player}
	elif ghost_type != 'genetic':
		raise ValueError(f"{ghost_type} is not a known type of ghost agent. Accepted types are ['wander','chase','genetic']")
	
	
	# game loop
	while not game.gameover:
		for player in game.players:
			if 'm' in player:
				# select Pac-Man move using either a genetic or a static agent
				if pac_type == 'genetic':
					actions = game.get_actions(player=player)
					s_primes = game.get_observations(actions, player=player)
					selected_action = None
					# YOUR PAC-MAN CODE GOES HERE##############################
					# TODO: score states stored in s_prime with pac_controller

					# TODO: assign index of state with the best score to selected_action

					# YOUR CODE (PROBABLY) ENDS HERE###########################
					# register selected action with game
					game.register_action(actions[selected_action], player=player)
				else:
					game.register_action(pac[player].select_action(game, player), player)
			else:
				# select ghost move using either a genetic or a static agent
				if ghost_type == 'genetic':
					actions = game.get_actions(player=player)
					s_primes = game.get_observations(actions, player=player)
					selected_action = None
					# YOUR GHOST CODE GOES HERE################################
					# TODO: score states stored in s_prime with ghost_controller

					# TODO: assign index of state with the best score to selected_action

					# YOUR CODE (PROBABLY) ENDS HERE###########################
					# register selected action with game
					game.register_action(actions[selected_action], player=player)
				else:
					game.register_action(ghosts[player].select_action(game, player), player)
		game.step()
	return game.score, game.log
```

# Side Note: Canonical Genetic Programming

Should you apply GP after this class, you should know that the GP algorithm taught in this class (as described in the course textbook) differs somewhat from the algorithm canonically used in GP. Notably, the textbook has certain important omissions regarding the Ramped Half-and-half algorithm and the GP evolutionary cycle. Canonically, the Ramped Half-and-half algorithm uses a `grow` method which ensures at least 1 branch reaches the depth limit. This can be difficult to implement, and has little impact on this assignment, so we don't require the implementation of the canonical version of the algorithm.

More importantly, however, is that the canonical GP evolutionary cycle is generational in nature. In the canonical Generational GP algorithm, $\mu$ children are created each generation via recombination, mutation, or reproduction and the children directly replace the parents without survival selection. The algorithm you implement for this assignment series is much more similar to a Genetic Algorithm in nature and you should be aware of this distinction if you continue to work with GP. For more information see [here](https://geneticprogramming.com/about-gp/gp-workflow/).